In [ ]:
import numpy as np
from astropy.io import fits
from astropy.table import Table
from photutils.aperture import CircularAperture, aperture_photometry
from astropy.stats import sigma_clipped_stats


In [ ]:
import matplotlib.pyplot as plt
import arya

In [ ]:
from astropy.nddata import CCDData

In [ ]:
import sys
sys.path.append("../")
sys.path.append("../../imaging")

from phot_utils import swap_byteorder

In [ ]:
from pathlib import Path

In [ ]:
from ccdproc import ImageFileCollection

In [ ]:
def get_catalogues(dirname, filtname):
    cats = []
    for folder in dirname.glob(f"img_{filtname}_*"):
        file = folder / "detection.cat"
        if file.is_file():
            cat = Table.read(file, hdu=2)
            cats.append(cat)

    return cats

In [ ]:
cats = get_catalogues(Path("../yasone2"), "i")

In [ ]:
cats[0]

In [ ]:
for cat in cats:
    plt.scatter(cat["ALPHA_J2000"], cat["DELTA_J2000"], s=1)

In [ ]:
cat.rename_columns?

In [ ]:
def new_names(cat, suffix):
    names = cat.columns
    return tuple(names), tuple([name + suffix for name in names])

In [ ]:
from phot_utils import xmatch

In [ ]:
?xmatch

In [ ]:
import astropy.units as u

In [ ]:
cat.columns

In [ ]:
def xmatch_all(cats):
    cat_all = cats[0].copy()

    cat_all["FLUX_0"] = cat_all["FLUX_APER"]
    cat_all["FLUX_ERR_0"] = cat_all["FLUXERR_APER"]
    
    filt_all = np.full(len(cat_all), True)
    for i in range(1, len(cats)):
        cat = cats[i]
        idxs, dists, filt = xmatch(cat_all["ALPHA_J2000"], cat_all["DELTA_J2000"], cat["ALPHA_J2000"], cat["DELTA_J2000"], 3*u.arcsec)
        filt_all &= filt
        cat_all[f"FLUX_{i}"] = cat["FLUX_APER"][idxs]
        cat_all[f"FLUX_ERR_{i}"] = cat["FLUXERR_APER"][idxs]

    return cat_all[filt_all]

In [ ]:
cat_all = xmatch_all(cats)

In [ ]:
len(cat_all), len(cats[0])

In [ ]:
cat_good = cat_all[cat_all["FLAGS"] == 0]

In [ ]:
plt.scatter(cat_good["ALPHA_J2000"], cat_good["DELTA_J2000"])

In [ ]:
allflux = np.stack([cat_good[f"FLUX_{i}"] for i in range(len(cats))])
allflux_err = np.stack([cat_good[f"FLUX_ERR_{i}"] for i in range(len(cats))])

In [ ]:
med_mag = np.median(allflux, axis=0)
std_mag = np.std(allflux, axis=0, ddof=1)
med_std = np.median(allflux_err, axis=0)

In [ ]:
Nap = allflux.shape[-1]

In [ ]:
for i in range(Nap):
    z = std_mag[:, i] / med_std[:, i]
    plt.scatter(med_mag[:, i], z, s=1, alpha=0.1, color=arya.COLORS[i])
    plt.xscale("log")
    plt.yscale("log")
    plt.scatter(np.median(med_mag[:, i]), np.median(z), color=arya.COLORS[i], ec="black", zorder=3, lw=0.3)
plt.axhline(1)
plt.axhline(2)
plt.xlabel("med flux")
plt.ylabel("mag std / median reported std")

In [ ]:
i = 3
fluxes = allflux[:, :, i]
fluxerrs = allflux_err[:, :, i]

z = (fluxes - np.median(fluxes, axis=0)) / fluxerrs


fig, axs = plt.subplots(2, 1, sharex=True, height_ratios=[0.3, 1], gridspec_kw={"hspace": 0}, figsize=(4, 4))
plt.sca(axs[1])
plt.scatter(z, fluxes, s=1, alpha=0.2, lw=0, color="k")
plt.yscale("log")
plt.xlim(-5, 5)
plt.xlabel("z score")
plt.ylabel("flux")

plt.sca(axs[0])
plt.hist(z.flatten(), np.linspace(-5, 5, 100), density=True)

x = np.linspace(-5, 5, 1000)
y = 1 / np.sqrt(2*np.pi) * np.exp(-x**2 / 2)
plt.plot(x, y, color=arya.COLORS[1])
plt.ylabel("density")

